### Load training datset

In [ ]:
!ls /content/drive/MyDrive/hackathon-nlp/training_dataset

df.json


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.6 MB/s 
     |████████████████████████████████| 212 kB 67.2 MB/s 
     |████████████████████████████████| 115 kB 69.5 MB/s 
     |████████████████████████████████| 120 kB 75.5 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import os, json
import pandas as pd

In [ ]:
columns_jsons_dataframe=['text', 'label']

In [ ]:
columns_jsons_dataframe_ts=['timestamp']

In [ ]:
jsons_dataframe = pd.DataFrame(columns=['text', 'label'])

In [ ]:
path_to_json = '/content/drive/MyDrive/hackathon-nlp/training_dataset'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('df.json')]
print(json_files[0])

df.json


In [ ]:
for index, js in enumerate(json_files):
  with open(os.path.join(path_to_json, js)) as json_files:
      json_texts = json.load(json_files)
      for json_text in json_texts:
        text = json_text['text']
        label = json_text['bug']
        df = pd.DataFrame([[text, label]], columns=columns_jsons_dataframe)
        jsons_dataframe = jsons_dataframe.append(df, ignore_index=True)

print(jsons_dataframe)


                                                  text label
0                       formations? sure. lets make a.     0
1                         eric thing i can experiment.     0
2    oh yeah and theres also so many weirdness issu...     1
3    so it ignores anything. let me try this again....     1
4                        it permits me to build right?     0
..                                                 ...   ...
410  and the right click on things was incredibly f...     1
411  before the fall celts among dlc. true hi to vi...     1
412  before the celts under dlc. true hello view. w...     1
413  the hp is not going any faster. they just stop...     1
414  the hp doesn't get faster. They just stop. The...     1

[415 rows x 2 columns]


In [ ]:
path_to_json = '/content/drive/MyDrive/hackathon-nlp/test-text-json'
test_json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('age4multiplayer.json')]
test_json = test_json_files[0]

In [ ]:
jsons_dataframe_test = pd.DataFrame(columns=['text', 'label'])

In [ ]:
jsons_dataframe_test_ts = pd.DataFrame(columns=['timestamp'])

In [ ]:
with open(os.path.join(path_to_json, test_json)) as test_json_files:
      json_texts = json.load(test_json_files)
      for json_text in json_texts:
        text = json_text['text']
        label = json_text['bug']
        timestamp = json_text['timestamp']
        df_test = pd.DataFrame([[text, label]], columns=columns_jsons_dataframe)
        df_test_ts = pd.DataFrame([[timestamp]], columns=columns_jsons_dataframe_ts)
        jsons_dataframe_test = jsons_dataframe_test.append(df_test, ignore_index=True)
        jsons_dataframe_test_ts = jsons_dataframe_test_ts.append(df_test_ts, ignore_index=True)

print(jsons_dataframe_test)
print(jsons_dataframe_test_ts)

                                                  text label
0                                         No go go go.     0
1                                               Right?     0
2    So English Chinese, Holy, Roman Empire and Abb...     0
3    Getting bills really feels like like getting m...     0
4    So for those of you from age of empires two, y...     0
..                                                 ...   ...
200                                                No.     0
201  Yeah, a little bit of a weird bit of a anticli...     0
202  But I didn't have quite enough stone for the f...     0
203                               Elite lands connect.     0
204  That whatever they have like the fire thing do...     0

[205 rows x 2 columns]
     timestamp
0        10.78
1        14.48
2        18.78
3        43.76
4        54.01
..         ...
200    1296.36
201    1297.86
202    1303.79
203    1317.39
204    1319.69

[205 rows x 1 columns]


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 


### Preprocess

##### Load the DistilBERT tokenizer to process the text field:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
jsons_dataframe

,text,label
0,formations? sure. lets make a.,0
1,eric thing i can experiment.,0
2,oh yeah and theres also so many weirdness issu...,1
3,so it ignores anything. let me try this again....,1
4,it permits me to build right?,0
...,...,...
410,and the right click on things was incredibly f...,1
411,before the fall celts among dlc. true hi to vi...,1
412,before the celts under dlc. true hello view. w...,1
413,the hp is not going any faster. they just stop...,1


##### Apply the function tokenizer to generate input_ids and attention_mask two columns

In [ ]:
jsons_dataframe['input_ids'] = jsons_dataframe.apply(lambda x: tokenizer(x["text"], truncation=True)['input_ids'], axis = 1)
jsons_dataframe['attention_mask'] = jsons_dataframe.apply(lambda x: tokenizer(x["text"], truncation=True)['attention_mask'], axis = 1)

In [ ]:
jsons_dataframe_test['input_ids'] = jsons_dataframe_test.apply(lambda x: tokenizer(x["text"], truncation=True)['input_ids'], axis = 1)
jsons_dataframe_test['attention_mask'] = jsons_dataframe_test.apply(lambda x: tokenizer(x["text"], truncation=True)['attention_mask'], axis = 1)

In [ ]:
jsons_dataframe

,text,label,input_ids,attention_mask
0,formations? sure. lets make a.,0,"[101, 13197, 1029, 2469, 1012, 11082, 2191, 10...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,eric thing i can experiment.,0,"[101, 4388, 2518, 1045, 2064, 7551, 1012, 102]","[1, 1, 1, 1, 1, 1, 1, 1]"
2,oh yeah and theres also so many weirdness issu...,1,"[101, 2821, 3398, 1998, 2045, 2015, 2036, 2061...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,so it ignores anything. let me try this again....,1,"[101, 2061, 2009, 26663, 2505, 1012, 2292, 203...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,it permits me to build right?,0,"[101, 2009, 14245, 2033, 2000, 3857, 2157, 102...","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...
410,and the right click on things was incredibly f...,1,"[101, 1998, 1996, 2157, 11562, 2006, 2477, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
411,before the fall celts among dlc. true hi to vi...,1,"[101, 2077, 1996, 2991, 8292, 21593, 2426, 214...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
412,before the celts under dlc. true hello view. w...,1,"[101, 2077, 1996, 8292, 21593, 2104, 21469, 22...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
413,the hp is not going any faster. they just stop...,1,"[101, 1996, 6522, 2003, 2025, 2183, 2151, 5514...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
jsons_dataframe_test

,text,label,input_ids,attention_mask
0,No go go go.,0,"[101, 2053, 2175, 2175, 2175, 1012, 102]","[1, 1, 1, 1, 1, 1, 1]"
1,Right?,0,"[101, 2157, 1029, 102]","[1, 1, 1, 1]"
2,"So English Chinese, Holy, Roman Empire and Abb...",0,"[101, 2061, 2394, 2822, 1010, 4151, 1010, 3142...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Getting bills really feels like like getting m...,0,"[101, 2893, 8236, 2428, 5683, 2066, 2066, 2893...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"So for those of you from age of empires two, y...",0,"[101, 2061, 2005, 2216, 1997, 2017, 2013, 2287...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
200,No.,0,"[101, 2053, 1012, 102]","[1, 1, 1, 1]"
201,"Yeah, a little bit of a weird bit of a anticli...",0,"[101, 3398, 1010, 1037, 2210, 2978, 1997, 1037...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
202,But I didn't have quite enough stone for the f...,0,"[101, 2021, 1045, 2134, 1005, 1056, 2031, 3243...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
203,Elite lands connect.,0,"[101, 7069, 4915, 7532, 1012, 102]","[1, 1, 1, 1, 1, 1]"


#### Transform the dataframe to dataset

In [ ]:
from datasets import Dataset
jsons_dataset_training = Dataset.from_pandas(jsons_dataframe)

In [ ]:
jsons_dataset_test = Dataset.from_pandas(jsons_dataframe_test)

In [ ]:
jsons_dataset_training

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 415
})

In [ ]:
jsons_dataset_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 205
})

### Train

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=jsons_dataset_training,
    eval_dataset=jsons_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 415
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 130
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=130, training_loss=0.4139535757211538, metrics={'train_runtime': 11.4267, 'train_samples_per_second': 181.593, 'train_steps_per_second': 11.377, 'total_flos': 31759317553164.0, 'train_loss': 0.4139535757211538, 'epoch': 5.0})

In [ ]:
test_results = trainer.predict(jsons_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 205
  Batch size = 16


In [ ]:
type(test_results[0])

numpy.ndarray

##### Transform the logits to probability

In [ ]:
from scipy.special import softmax

probabilities = softmax(test_results[0], axis=1)

In [ ]:
probabilities

In [ ]:
test_results_prob = pd.DataFrame(probabilities, columns=['NEG', 'POS'])

In [ ]:
test_results_prob

In [ ]:
import numpy as np

In [ ]:
test_results_prob['prediction'] = np.where(test_results_prob['POS'] > 0.996, 1, 0)

In [ ]:
test_results_prob

,NEG,POS,prediction
0,0.997609,0.002391,0
1,0.997695,0.002305,0
2,0.003174,0.996826,1
3,0.003738,0.996262,1
4,0.955896,0.044104,0
...,...,...,...
200,0.997051,0.002949,0
201,0.003684,0.996316,1
202,0.002442,0.997558,1
203,0.996638,0.003362,0


In [ ]:
results_dataframe = jsons_dataframe_test.join(test_results_prob)

In [ ]:
results_dataframe

,text,label,input_ids,attention_mask,NEG,POS,prediction
0,No go go go.,0,"[101, 2053, 2175, 2175, 2175, 1012, 102]","[1, 1, 1, 1, 1, 1, 1]",0.997609,0.002391,0
1,Right?,0,"[101, 2157, 1029, 102]","[1, 1, 1, 1]",0.997695,0.002305,0
2,"So English Chinese, Holy, Roman Empire and Abb...",0,"[101, 2061, 2394, 2822, 1010, 4151, 1010, 3142...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.003174,0.996826,1
3,Getting bills really feels like like getting m...,0,"[101, 2893, 8236, 2428, 5683, 2066, 2066, 2893...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.003738,0.996262,1
4,"So for those of you from age of empires two, y...",0,"[101, 2061, 2005, 2216, 1997, 2017, 2013, 2287...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.955896,0.044104,0
...,...,...,...,...,...,...,...
200,No.,0,"[101, 2053, 1012, 102]","[1, 1, 1, 1]",0.997051,0.002949,0
201,"Yeah, a little bit of a weird bit of a anticli...",0,"[101, 3398, 1010, 1037, 2210, 2978, 1997, 1037...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.003684,0.996316,1
202,But I didn't have quite enough stone for the f...,0,"[101, 2021, 1045, 2134, 1005, 1056, 2031, 3243...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.002442,0.997558,1
203,Elite lands connect.,0,"[101, 7069, 4915, 7532, 1012, 102]","[1, 1, 1, 1, 1, 1]",0.996638,0.003362,0


In [ ]:
actual = results_dataframe['label'].to_numpy()
predicted = results_dataframe['prediction'].to_numpy()

In [ ]:
print(jsons_dataframe_test)
print(jsons_dataframe_test_ts)

In [ ]:
predicted_result = pd.DataFrame(predicted_array, columns=['predicted'])

In [ ]:
predicted_join_result = jsons_dataframe_test.join(jsons_dataframe_test_ts).join(predicted_result)

In [ ]:
predicted_join_result.to_csv('predicted_join_result.csv')

### Model Evaluation

##### V2 model

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
print(tn, fp, fn, tp)

177 22 0 6


In [ ]:
precision = tp / (tp+fp)

In [ ]:
recall = tp / (tp+fn)

In [ ]:
precision

0.21428571428571427

In [ ]:
recall

1.0

In [ ]:
accuracy = (tp + tn) / 205
accuracy

0.8926829268292683

V1 model

In [ ]:
from sklearn.metrics import confusion_matrix
tn_p, fp_p, fn_p, tp_p = confusion_matrix([1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                                  [1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).ravel()
print(tn_p, fp_p, fn_p, tp_p)

189 10 4 2


In [ ]:
precision_p = tp_p / (tp_p + fp_p)
recall_p = tp_p / (tp_p + fn_p)
accuracy_p = (tp_p + tn_p) / 205
precision

0.16666666666666666

In [ ]:
recall_p

0.3333333333333333

In [ ]:
accuracy_p

0.9317073170731708